<a href="https://colab.research.google.com/github/alievk/avatarify/blob/master/avatarify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Avatarify Colab Server

This Colab notebook is for running Avatarify rendering server. It allows you to run Avatarify on your computer **without GPU** in this way:

1. When this notebook is executed, it starts listening for incoming requests from your computer;
1. You start the client on your computer and it connects to the notebook and starts sending requests;
1. This notebooks receives the requests from your computer, renders avatar images and sends them back;

To this end, all the heavy work is offloaded from your computer to this notebook so you don't need to have a beafy hardware on your PC anymore.


## Start the server
Run the cells below (Shift+Enter) sequentially and pay attention to the hints and instructions included in this notebook.

At the end you will get a command for running the client on your computer.

## Start the client

Make sure you have installed the latest version of Avatarify on your computer. Refer to the README for the instructions.

When it's ready execute this notebook and get the command for running the client on your computer.


### Technical details

The client on your computer connects to the server via `ngrok` TCP tunnel or a reverse `ssh` tunnel.

`ngrok`, while easy to use, can induce a considerable network lag ranging from dozens of milliseconds to a second. This can lead to a poor experience.

A more stable connection could be established using a reverse `ssh` tunnel to a host with a public IP, like an AWS `t3.micro` (free) instance. This notebook provides a script for creating a tunnel, but launching an instance in a cloud is on your own (find the manual below).

# Install

### Avatarify
Follow the steps below to clone Avatarify and install the dependencies.

In [0]:
!cd /content
!rm -rf *

In [2]:
!git clone https://github.com/alievk/avatarify.git

Cloning into 'avatarify'...
remote: Enumerating objects: 64, done.
remote: Counting objects: 100% (64/64), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 1113 (delta 38), reused 43 (delta 23), pack-reused 1049
Receiving objects: 100% (1113/1113), 5.55 MiB | 4.40 MiB/s, done.
Resolving deltas: 100% (705/705), done.


In [3]:
cd avatarify

/content/avatarify


In [4]:
!git clone https://github.com/alievk/first-order-model.git fomm
!pip install face-alignment==1.0.0 msgpack_numpy

Cloning into 'fomm'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 211 (delta 3), reused 6 (delta 2), pack-reused 200
Receiving objects: 100% (211/211), 58.17 MiB | 10.31 MiB/s, done.
Resolving deltas: 100% (102/102), done.


In [5]:
!scripts/download_data.sh

Getting cookie
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   408    0   408    0     0   1378      0 --:--:-- --:--:-- --:--:--  1378
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
100  228M    0  228M    0     0  28.4M      0 --:--:--  0:00:08 --:--:-- 51.7M
Expected checksum: 8a45a24037871c045fbb8a6a8aa95ebc
Found checksum:    8a45a24037871c045fbb8a6a8aa95ebc  vox-adv-cpk.pth.tar


### ngrok
Follow the steps below to setup ngrok. You will also need to sign up on the ngrok site and get your authtoken (free).


In [6]:
# Download ngrok
!scripts/get_ngrok.sh

ngrok is not found, installing...
Done!


# Run
Start here if the runtime was restarted after installation.

In [7]:
cd /content/avatarify

/content/avatarify


In [0]:
#!git pull origin

In [0]:
from subprocess import Popen, PIPE
import shlex
import json
import time


def run_with_pipe(command):
  commands = list(map(shlex.split,command.split("|")))
  ps = Popen(commands[0], stdout=PIPE, stderr=PIPE)
  for command in commands[1:]:
    ps = Popen(command, stdin=ps.stdout, stdout=PIPE, stderr=PIPE)
  return ps.stdout.readlines()


def get_tunnel_adresses():
  info = run_with_pipe("curl http://localhost:4040/api/tunnels")
  assert info

  info = json.loads(info[0])
  for tunnel in info['tunnels']:
    url = tunnel['public_url']
    port = url.split(':')[-1]
    local_port = tunnel['config']['addr'].split(':')[-1]
    print(f'{url} -> {local_port} [{tunnel["name"]}]')
    if tunnel['name'] == 'input':
      in_addr = url
    elif tunnel['name'] == 'output':
      out_addr = url
    else:
      print(f'unknown tunnel: {tunnel["name"]}')

  return in_addr, out_addr

In [0]:
# Input and output ports for communication
local_in_port = 5557
local_out_port = 5558

# Start the worker


In [0]:
# (Re)Start the worker
with open('/tmp/run.txt', 'w') as f:
  ps = Popen(
      shlex.split(f'./run.sh --is-worker --in-port {local_in_port} --out-port {local_out_port} --no-vcam --no-conda'),
      stdout=f, stderr=f)
  time.sleep(3)

This command should print lines if the worker is successfully started

In [40]:
!ps aux | grep 'python3 afy/cam_fomm.py' | grep -v grep | tee /tmp/ps_run
!if [[ $(cat /tmp/ps_run | wc -l) == "0" ]]; then echo "Worker failed to start"; cat /tmp/run.txt; else echo "Worker started"; fi

root        1114 57.2  2.1 2063012 283120 ?      S    13:55   0:02 python3 afy/cam_fomm.py --config fomm/config/vox-adv-256.yaml --checkpoint vox-adv-cpk.pth.tar --cam 0 --virt-cam 9 --relative --adapt_scale --is-worker --in-port 5557 --out-port 5558 --no-stream
root        1126  0.0  1.1 2071208 152352 ?      Sl   13:55   0:00 python3 afy/cam_fomm.py --config fomm/config/vox-adv-256.yaml --checkpoint vox-adv-cpk.pth.tar --cam 0 --virt-cam 9 --relative --adapt_scale --is-worker --in-port 5557 --out-port 5558 --no-stream
root        1129  0.0  1.1 2063012 149412 ?      S    13:55   0:00 python3 afy/cam_fomm.py --config fomm/config/vox-adv-256.yaml --checkpoint vox-adv-cpk.pth.tar --cam 0 --virt-cam 9 --relative --adapt_scale --is-worker --in-port 5557 --out-port 5558 --no-stream
root        1130  0.0  1.1 2071208 152464 ?      Sl   13:55   0:00 python3 afy/cam_fomm.py --config fomm/config/vox-adv-256.yaml --checkpoint vox-adv-cpk.pth.tar --cam 0 --virt-cam 9 --relative --adapt_scale --i

# Open ngrok tunnel

#### Get ngrok token
Go to https://dashboard.ngrok.com/auth/your-authtoken (sign up if required), copy your authtoken and put it below.

In [0]:
# Paste your authtoken here in quotes
authtoken = "1cBzFFwzSlaLhlRPXIHJiVLqtiQ_2cVsonJXe52B6DDyp8su8"

Set your region

Code | Region
--- | ---
us | United States
eu | Europe
ap | Asia/Pacific
au | Australia
sa | South America
jp | Japan
in | India

In [0]:
# Set your region here in quotes
region = "eu"

In [0]:
config =\
f"""
authtoken: {authtoken}
region: {region}
console_ui: False
tunnels:
  input:
    addr: {local_in_port}
    proto: tcp    
  output:
    addr: {local_out_port}
    proto: tcp
"""

with open('ngrok.conf', 'w') as f:
  f.write(config)

In [0]:
# (Re)Open tunnel
ps = Popen('./scripts/open_tunnel_ngrok.sh', stdout=PIPE, stderr=PIPE)
time.sleep(3)

In [31]:
# Get tunnel addresses
try:
  in_addr, out_addr = get_tunnel_adresses()
  print("Tunnel opened")
except Exception as e:
  [print(l.decode(), end='') for l in ps.stdout.readlines()]
  print("Something went wrong, reopen the tunnel")

tcp://2.tcp.eu.ngrok.io:19858 -> 5558 [output]
tcp://2.tcp.eu.ngrok.io:14088 -> 5557 [input]
Tunnel opened


### [Optional] AWS proxy
Alternatively you can create a ssh reverse tunnel to an AWS `t3.micro` instance (it's free). It has lower latency than ngrok.

1. In your AWS console go to Services -> EC2 -> Instances -> Launch Instance;
1. Choose `Ubuntu Server 18.04 LTS` AMI;
1. Choose `t3.micro` instance type and press Review and launch;
1. Confirm your key pair and press Launch instances;
1. Go to the security group of this instance and edit inbound rules. Add TCP ports 5557 and 5558 and set Source to Anywhere. Press Save rules;
1. ssh into the instance (you can find the command in the Instances if you click on the Connect button) and add this line in the end of `/etc/ssh/sshd_config`:
```
GatewayPorts yes
```
then restart `sshd`
```
sudo service sshd restart
```
1. Copy your `key_pair.pem` by dragging and dropping it into avatarify folder in this notebook;
1. Use the command below to open the tunnel;
1. Start client with a command (substitute `run_mac.sh` with `run_windows.bat` or `run.sh`)
```
./run_mac.sh --is-client --in-addr tcp://instace.compute.amazonaws.com:5557 --out-addr tcp://instance.compute.amazonaws.com:5558
```

In [0]:
# Open reverse ssh tunnel (uncomment line below)
# !./scripts/open_tunnel_ssh.sh key_pair.pem ubuntu@instance.compute.amazonaws.com

# Start the client
When you run the cell below it will print a command. Copy-paste and run this command in you computer's terminal.

In [43]:
print('Mac:')
print(f'./run_mac.sh --is-client --in-addr {in_addr} --out-addr {out_addr}')
print('\nWindows:')
print(f'run_windows.bat --is-client --in-addr {in_addr} --out-addr {out_addr}')
print('\nLinux:')
print(f'./run.sh --is-client --in-addr {in_addr} --out-addr {out_addr}')

Mac:
./run_mac.sh --is-client --in-addr tcp://2.tcp.eu.ngrok.io:14088 --out-addr tcp://2.tcp.eu.ngrok.io:19858

Windows:
run_windows.bat --is-client --in-addr tcp://2.tcp.eu.ngrok.io:14088 --out-addr tcp://2.tcp.eu.ngrok.io:19858

Linux:
./run.sh --is-client --in-addr tcp://2.tcp.eu.ngrok.io:14088 --out-addr tcp://2.tcp.eu.ngrok.io:19858


# Logs

If something doesn't work as expected, please run the cells below and include the logs in your report.

In [33]:
!cat ./var/log/cam_fomm.log | head -100

[1590155135.308746] Loading Predictor


In [34]:
!cat ./var/log/recv_worker.log | tail -100

[1590155447.549979] recv timeout
[1590155448.551291] recv timeout
[1590155449.552548] recv timeout
[1590155450.553864] recv timeout
[1590155451.555094] recv timeout
[1590155452.556346] recv timeout
[1590155453.557606] recv timeout
[1590155454.558854] recv timeout
[1590155455.560134] recv timeout
[1590155456.561301] recv timeout
[1590155457.562503] recv timeout
[1590155458.563707] recv timeout
[1590155459.564918] recv timeout
[1590155460.565634] recv timeout
[1590155461.565916] recv timeout
[1590155462.567144] recv timeout
[1590155463.568523] recv timeout
[1590155464.569717] recv timeout
[1590155465.570982] recv timeout
[1590155466.572245] recv timeout
[1590155467.573463] recv timeout
[1590155468.574726] recv timeout
[1590155469.575710] recv timeout
[1590155470.576911] recv timeout
[1590155471.578154] recv timeout
[1590155472.579465] recv timeout
[1590155473.580739] recv timeout
[1590155474.581996] recv timeout
[1590155475.583220] recv timeout
[1590155476.584012] recv timeout
[159015547

In [0]:
!cat ./var/log/predictor_worker.log | tail -100

In [42]:
!cat ./var/log/send_worker.log | tail -100

[1590155721.628546] Sending on port 5558
[1590155722.630178] send queue empty
[1590155723.630507] send queue empty
[1590155724.631508] send queue empty
[1590155725.632688] send queue empty
[1590155726.633851] send queue empty
[1590155727.635052] send queue empty
[1590155728.636242] send queue empty
[1590155729.637451] send queue empty
[1590155730.638657] send queue empty
[1590155731.639494] send queue empty
[1590155732.640535] send queue empty
[1590155733.641758] send queue empty
[1590155734.642961] send queue empty
[1590155735.644146] send queue empty
[1590155736.645349] send queue empty
[1590155737.646558] send queue empty
[1590155738.647744] send queue empty
[1590155739.648923] send queue empty
[1590155740.650120] send queue empty
[1590155741.651335] send queue empty
[1590155742.652588] send queue empty
[1590155743.653787] send queue empty
[1590155744.654958] send queue empty
[1590155745.656144] send queue empty
[1590155746.657313] send queue empty
[1590155747.658501] send queue emp